<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

_____

<a id='home'></a>

# Reshaping

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/reshaping/blob/main/Reshaping.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



Data frames have have different shapes. Let me use some columns from the data on fragility:

In [ ]:
import os, pandas as pd

FragilityAll=pd.read_csv('https://raw.githubusercontent.com/CienciaDeDatosEspacial/appending/main/Fragility.csv')

#
FragilityAll.head()

Let me keep some variables:

In [ ]:
keep=['Country','Year','C1_SecurityApparatus','C2_FactionalizedElites','C3_GroupGrievance']
FragilitySome=FragilityAll.loc[:,keep].copy()

This is a **long** shape:

In [ ]:
# country names will repeat:
FragilitySome.sort_values(by='Country')

But, if I choose one year, I have a wide shape:

In [ ]:
FragilitySome[FragilitySome.Year==2013]

## From long to wide

In [ ]:
# to wide
FragilitySome_W=pd.pivot_table(FragilitySome,
                               index=['Country'], # unit of analysis
                               columns=['Year']) # the values for NEW column

FragilitySome_W

Notice the column names:

In [ ]:
FragilitySome_W.columns

In [ ]:
#then
FragilitySome_W.columns=['.'.join([col[0],str(col[1])]) for col in FragilitySome_W.columns]
FragilitySome_W.head()

As usual, you want the index as a column:

In [ ]:
FragilitySome_W.reset_index(drop=False,inplace=True)
FragilitySome_W

Long shape is efficient in the presence of missing values:

In [ ]:
# missing values in long format
FragilitySome[FragilitySome.isna().any(axis=1)]

Wide shape will create missing values when needed:

In [ ]:
# what cells have missing values?
FragilitySome_W[FragilitySome_W.isna().any(axis=1)]

This last request shows us problems.
As you can see, even though the data DFs were prepared by the same organization, the DFs have country names that differ among them. Here we need some **manual** changes:

In [ ]:
# prepare changes as dict:
changes={"CABO VERDE": "CAPE VERDE",
"CZECHIA":"CZECH REPUBLIC",
"SWAZILAND":"ESWATINI",
"ISRAEL AND WEST BANK":"ISRAEL",
"KYRGYZSTAN":"KYRGYZ REPUBLIC",
"NORTH MACEDONIA":"MACEDONIA",
"SLOVAKIA": "SLOVAK REPUBLIC"}

Notice I am making the changes in the long shape of the DF, so that the wide shape will work fine:

In [ ]:
# make changes using 'replace':
FragilityAll.replace(to_replace={'Country':changes},inplace=True)

# re saving

FragilityAll.to_csv('Fragility_cleaner.csv',index=False)

#then
FragilitySome=FragilityAll.loc[:,keep].copy()

Let's redo the wide reshape:

In [ ]:
# to wide shape again
FragilitySome_W=pd.pivot_table(FragilitySome,
                               index=['Country'], # unit of analysis
                               columns=['Year']) # the values for NEW column


FragilitySome_W.columns=['.'.join([col[0],str(col[1])]) for col in FragilitySome_W.columns]
# verify missing
FragilitySome_W[FragilitySome_W.isna().any(axis=1)]

You have to be very careful when working with countries, specially when you are including or excluding countries; which may cause you hurting someone else's feelings.

For instance, here I am just keeping **rows** with no missing values:

In [ ]:
FragilitySome_W.dropna(inplace=True)
FragilitySome_W

## From wide to long

In [ ]:
# we had
FragilitySome2013=FragilitySome[FragilitySome.Year==2013]
FragilitySome2013

In [ ]:
#then
pd.melt(FragilitySome2013,id_vars=['Country','Year'])

...and back to the original:

In [ ]:
FragilitySome2013_L=pd.melt(FragilitySome2013,id_vars=['Country','Year'])

# to wide
pd.pivot_table(FragilitySome2013_L,
               index=['Country','Year'], # unit of analysis
               columns=['variable']) # the values for NEW column

This looks more complex:

In [ ]:
FragilitySome_W

The strategy (for pandas) will be to re create the multi index:

In [ ]:
# currently:
FragilitySome_W.columns

In [ ]:
# new cols
[tuple(col.split('.')) for col in FragilitySome_W.columns]

In [ ]:
# making changes:

colTuples=[tuple(col.split('.')) for col in FragilitySome_W.columns]
pd.MultiIndex.from_tuples(colTuples, names=('variable', 'year'))

Using the multiindex:

In [ ]:
FragilitySome_W.columns=pd.MultiIndex.from_tuples(colTuples, names=('measure', 'year'))
FragilitySome_W

In this situation, melting is easy:

In [ ]:
FragilitySome_W.melt(ignore_index = False)